In [5]:
import time
start_time = time.time()

In [3]:
import pandas as pd
import numpy as np
import re
import string
import spacy

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB

from sklearn.metrics import classification_report, accuracy_score, recall_score
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV, KFold, cross_val_score, train_test_split, StratifiedKFold, GridSearchCV

import matplotlib.pyplot as plt

In [4]:
pd.set_option('max_colwidth', 400)

In [5]:
!python3 -m spacy download en_core_web_sm

2023-10-01 13:47:10.837863: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 40.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# Загрузка датасета "Spam or not spam":

In [7]:
data = pd.read_csv("/content/spam_or_not_spam.csv")
data.style.set_properties(**{'text-align': 'left'})
data.head(5)

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMBER NUMBER from chris garrigues cwg dated NUMBER NUMBERfaNUMBERd deepeddy com message id NUMBER NUMBER tmda deepeddy vircio com i can t reproduce this error for me it is very repeatable like every time without fail this is the debug log of the pick happening NUMBER NUMBER NUMBER pick_it exec pick inbox list lbrace lbrace subject ftp rbrace rbrace NU...,0
1,martin a posted tassos papadopoulos the greek sculptor behind the plan judged that the limestone of mount kerdylio NUMBER miles east of salonika and not far from the mount athos monastic community was ideal for the patriotic sculpture as well as alexander s granite features NUMBER ft high and NUMBER ft wide a museum a restored amphitheatre and car park for admiring crowds are planned so is thi...,0
2,man threatens explosion in moscow thursday august NUMBER NUMBER NUMBER NUMBER pm moscow ap security officers on thursday seized an unidentified man who said he was armed with explosives and threatened to blow up his truck in front of russia s federal security services headquarters in moscow ntv television reported the officers seized an automatic rifle the man was carrying then the man got out...,0
3,klez the virus that won t die already the most prolific virus ever klez continues to wreak havoc andrew brandt from the september NUMBER issue of pc world magazine posted thursday august NUMBER NUMBER the klez worm is approaching its seventh month of wriggling across the web making it one of the most persistent viruses ever and experts warn that it may be a harbinger of new viruses that use a ...,0
4,in adding cream to spaghetti carbonara which has the same effect on pasta as making a pizza a deep pie i just had to jump in here as carbonara is one of my favourites to make and ask what the hell are you supposed to use instead of cream i ve never seen a recipe that hasn t used this personally i use low fat creme fraiche because it works quite nicely but the only time i ve seen an supposedly...,0


Оценим баланс классов:

In [8]:
data.label.value_counts()

0    2500
1     500
Name: label, dtype: int64

Загрузка набора английских стоп-слов из библиотеки Spacy:

In [9]:
nlp = spacy.load("en_core_web_sm")
stopwords = nlp.Defaults.stop_words
print(f'Spacy english stopwords size: {len(stopwords)}', end='\n\n')
', '.join(stopwords)

Spacy english stopwords size: 326



"during, take, amongst, almost, 'm, also, former, with, before, of, i, seemed, mine, nine, least, enough, 'd, without, or, anyhow, empty, go, hundred, everyone, but, still, an, n’t, must, after, afterwards, via, becoming, against, be, can, they, nothing, about, ever, around, one, four, ‘re, whole, should, something, per, three, the, namely, yet, hereafter, every, not, various, serious, thereby, much, are, further, same, ten, whoever, none, six, well, most, alone, what, into, next, its, amount, there, towards, fifty, keep, whenever, did, yourselves, quite, re, doing, you, ours, becomes, anyway, say, whither, wherein, beforehand, such, side, very, hers, because, beside, he, ‘ll, otherwise, whereafter, ’m, get, hereby, any, eleven, another, do, between, top, my, is, due, over, does, therein, eight, by, fifteen, may, beyond, herein, sometime, cannot, him, nowhere, few, moreover, n‘t, your, these, made, been, anyone, out, throughout, than, really, meanwhile, for, whence, sometimes, mostly, 

Удаление пустых и Nan строк из датасета:

In [10]:
data.drop(data[data.email == ' '].index, inplace=True)
data.drop(data[data['email'].isnull()].index, inplace=True)

Лемматизация и токенизация при помощи Spacy:

In [11]:
%%time
data['cleaned_text'] = data['email'].str.replace('NUMBER', '') # удаляем слово "NUMBER" из текста, так как оно заменяет собой все числа и встречается чаще любого другого слова
data['cleaned_text'] = data['cleaned_text'].apply(lambda x: ' '.join(token.lemma_.lower() for token in nlp(x) if
        not token.is_stop
        and not token.is_punct # удаление пунктуации
        and not token.is_digit # удаление цифр
        and not token.like_email # удаление почтовых адресов
        and not token.like_num # удаление чисел, в том числе в виде текста
        and not token.is_space # удаление пробельных символов
    )
)

data.sample(3, random_state=1)

CPU times: user 2min 27s, sys: 1.31 s, total: 2min 28s
Wall time: 2min 44s


,email,label,cleaned_text
748,at NUMBER NUMBER pm NUMBER on NUMBER NUMBER NUMBER tom wrote a green once said that if the spotted owl hadn t existed they would have had to invent it a republican once said i am not a crook oh great another round of lableisms let me know when you get back to real data not me i have another one instead green red cheers rah r a hettinga mailto rah ibuc com the internet bearer underwriting corpo...,0,pm tom write green say spot owl hadn t exist invent republican say crook oh great round lableism let know real datum instead green red cheer rah r hettinga mailto rah ibuc com internet bearer underwriting corporation url farquhar street boston ma usa deserve respect usefulness antiquity predict end world find agreeable experience edward gibbon decline fall roman empire
2883,pocket the newest NUMBER year annuity pocket the newest NUMBER year annuity from safe harbor financial NUMBER guaranteed for NUMBER years effective yield NUMBER year surrender period please fax back contracts with a copy of your insurance license to NUMBER NUMBER NUMBER click here to contract URL issued to age NUMBER a rated company alternative NUMBER year guaranteed plan call today for more ...,1,pocket new year annuity pocket new year annuity safe harbor financial guarantee year effective yield year surrender period fax contract copy insurance license click contract url issue age rate company alternative year guarantee plan today information fill form information address city state zip phone fax e mail don t want receive mailing wish receive professional communication send insurance p...
1391,justin mason jm jmason org NUMBER NUMBER NUMBER NUMBER NUMBER NUMBER NUMBER well my point is more that we should aim our rescoring algorithm so that a spam hits NUMBER NUMBER any higher does us no good as it means an fp is a lot harder to recover from using compensation rules agreed but i have always thought that the value NUMBER was not the best value it should have been NUMBER i understand t...,0,justin mason jm jmason org point aim rescore algorithm spam hit high good mean fp lot hard recover compensation rule agree think value good value understand initially spammy score include believe algorithm purely symmetrical non spammy negative value balance spammy positive value like sa today positive spam negative non spam guess exactly grey today s choice add offset think cause people assum...


Разобьем выборку на тренировочную и тестовую часть, зафиксируем random_state:

In [40]:
X_train, X_test, y_train, y_test = train_test_split(data['cleaned_text'], data['label'], shuffle=True, random_state=1234, train_size=0.7)

# Векторизация при помощи CountVectorizer

In [41]:
vectorizer = CountVectorizer()

X_train_bow_vectorized = vectorizer.fit_transform(X_train)
X_test_bow_vectorized = vectorizer.transform(X_test)

In [42]:
len(vectorizer.get_feature_names_out())

24072

In [43]:
vectorizer.get_feature_names_out()[2550:2650]

array(['bring', 'brique', 'britain', 'britan', 'britannia', 'british',
       'britney', 'briton', 'britons', 'brklisttmgepveytvbet', 'brksalrq',
       'brmodyyfnrlnvopxxmo', 'broad', 'broadband', 'broadcast',
       'broadcaster', 'broadcasting', 'broaden', 'broadening', 'broadly',
       'broadsheet', 'broadway', 'brocard', 'brochure', 'brochureware',
       'broil', 'broinn', 'broken', 'broker', 'brokerage', 'bromide',
       'bronson', 'bronze', 'broo', 'brooklyn', 'brooks', 'broom',
       'brose', 'brother', 'brotherhood', 'brotherton', 'brough', 'brown',
       'browse', 'browser', 'browsing', 'brruexlswclc', 'bruce', 'bruceg',
       'bruise', 'brunei', 'brunet', 'bruno', 'brunswick', 'brush',
       'brussels', 'brutal', 'brutality', 'brutalize', 'brutally',
       'brute', 'bryan', 'bs', 'bsbdfcrxnkmlt', 'bsd', 'bsddb', 'bsh',
       'bsmtp', 'bsp', 'bspyjykcbbe', 'bssejvecagazaaadhbuvqdkqomklnhix',
       'bssqftuds', 'bst', 'bstringfield',
       'bsuvzlvftsylsxzfqsttymckk

Представление текстов в виде мешка слов:

In [44]:
bag_count_vect = pd.DataFrame(X_train_bow_vectorized.toarray(), columns=vectorizer.get_feature_names_out())

,a_block,aa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaacuxrmplffhxl,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaacmmvzcjzld,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaacuwlcuwmdlo,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaabcfudhgkxt,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaacuxrmplffhxlmh,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaafcvwyfk,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaacuxrmplffhxlmhyv,...,허락없이,헤어디자이너,현재,호황을,홈쇼핑의,확실한,활황을,훨씬,힘입어,ｉt的技
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Векторизация при помощи TfidfVectorizer

In [45]:
tfidf_vectorizer = TfidfVectorizer()

X_train_tfidf_vectorized = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf_vectorized = tfidf_vectorizer.transform(X_test)

In [46]:
tfidf_vectorizer.get_feature_names_out()[1550:1650]

array(['avoir', 'avoisinante', 'avow',
       'avqcntxrhbjdjqipajpzxfacapkwkxqqvzs',
       'avrzlnrjrgslmdrwavgcnrwinhflritlzp', 'avyncusmrtgtlwlfkbyfn',
       'aw', 'await', 'awake', 'award', 'aware', 'awareness', 'away',
       'awcbwunahmxo', 'awdgvlaxitmbe', 'awe', 'awesome', 'awful',
       'awfully', 'awhile', 'awk', 'awkpxuovkyi',
       'awkuolcaljxrjlmlapmntvbpk', 'awkward', 'awl',
       'awubpwmbspxhjfohaeqlqgcgxhqeeuesnvuwgvmjfksanrb',
       'awubpxnfowyzbxegeqlfnwcfaknapokbgjjqqqehgiwfdsaopj',
       'awubpxzdpxhjfohaeqiaacgvmtbvktprjmjqtoyzyifzusan',
       'awubpycpxhjfohaeqlasgcfzhsqmsvuygqjwgldwzkpihmanrur',
       'awubpyyyspxhjfohaeqlqnqcgyvfnvywkirrghhdisbonconaopcr',
       'awubpzayrijkhjbjytpqeqiyzgcg', 'awvsuujywwmnpqrjwqmjplyvtqzhzzhk',
       'axe', 'axel', 'axf', 'axfr', 'axlbjrrsyyrtxoyrcuybydztzzxeos',
       'axle', 'axpgx', 'axtfrpxfcmuwyaljocwfir',
       'axwvyalyzqcyzkaplpturlkltbljy',
       'axycduvaomverzvrvzjsizbryjxwaqkdovuxtjxhvusxiif', 'axzn'

In [47]:
bag_tfidf_vect = pd.DataFrame(X_train_tfidf_vectorized.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

,a_block,aa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaacuxrmplffhxl,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaacmmvzcjzld,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaacuwlcuwmdlo,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaabcfudhgkxt,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaacuxrmplffhxlmh,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaafcvwyfk,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaacuxrmplffhxlmhyv,...,허락없이,헤어디자이너,현재,호황을,홈쇼핑의,확실한,활황을,훨씬,힘입어,ｉt的技
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Посмотрим на TF-IDF слова "yes":

In [48]:
bag_tfidf_vect[bag_tfidf_vect['yes'] > 0].sort_values(by=["yes"], ascending=False)['yes']

1702    0.183271
1738    0.161631
237     0.151156
207     0.139732
1778    0.133211
          ...   
964     0.010084
1745    0.010070
1305    0.009293
1285    0.005142
1565    0.003704
Name: yes, Length: 170, dtype: float64

In [49]:
bag_tfidf_vect.shape, bag_count_vect.shape, data.shape

((2097, 24072), (2097, 24072), (2997, 3))

# Кросс-валидация:

In [50]:
classifiers = [DecisionTreeClassifier(),
               LogisticRegression(),
               BernoulliNB()]

pipe_dict = {0: "DecisionTreeClassifier", 1: "LogisticRegression", 2: "Naive Bayes"}

Оценки для различных моделей и способов векторизации:

In [51]:
print('CountVectorizer')
for i, model in enumerate(classifiers):
    cv_score = cross_val_score(model, X_train_bow_vectorized, y_train, scoring="accuracy", cv=5)
    print("%s: %f " % (pipe_dict[i], cv_score.mean()))

CountVectorizer
DecisionTreeClassifier: 0.955646 
LogisticRegression: 0.981877 
Naive Bayes: 0.865995 


In [52]:
print("TfidfVectorizer")
for i, model in enumerate(classifiers):
    cv_score = cross_val_score(model, X_train_tfidf_vectorized, y_train, scoring="accuracy", cv=5)
    print("%s: %f " % (pipe_dict[i], cv_score.mean()))

TfidfVectorizer
DecisionTreeClassifier: 0.956131 
LogisticRegression: 0.943256 
Naive Bayes: 0.865995 


Видно, что наилучшую точность показывает LogisticRegression.

## Обучение моделей:

Выполним подбор гипер-параметров через пайплайн сразу для векторизации при помощи CountVectorizer и моделей:

### LR + CountVectorizer

In [53]:
pipe_counter_lr = Pipeline(
    steps=[
        ('counter', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', LogisticRegression())
    ]
)

In [54]:
parameter_grid = {'counter__max_df': np.arange(0.5, 1, 0.1),
                  'counter__min_df': np.arange(0.001, 0.01, 0.001),
                  "counter__ngram_range": ((1, 1), (1, 2)),
                  "tfidf__norm": ("l1", "l2"),
                  "clf__C": [0.001, 0.01, 0.1, 1, 10, 100]}

In [59]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2786)

In [60]:
%%time

grid_search_counter_lr = HalvingGridSearchCV(
    pipe_counter_lr,
    param_grid=parameter_grid,
    n_jobs=-1,
    verbose=1,
    cv=skf,
    scoring='accuracy',
    random_state=42,
)

grid_search_counter_lr.fit(X_train, y_train);

n_iterations: 5
n_required_iterations: 7
n_possible_iterations: 5
min_resources_: 20
max_resources_: 2097
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 1080
n_resources: 20
Fitting 5 folds for each of 1080 candidates, totalling 5400 fits
----------
iter: 1
n_candidates: 360
n_resources: 60
Fitting 5 folds for each of 360 candidates, totalling 1800 fits
----------
iter: 2
n_candidates: 120
n_resources: 180
Fitting 5 folds for each of 120 candidates, totalling 600 fits
----------
iter: 3
n_candidates: 40
n_resources: 540
Fitting 5 folds for each of 40 candidates, totalling 200 fits
----------
iter: 4
n_candidates: 14
n_resources: 1620
Fitting 5 folds for each of 14 candidates, totalling 70 fits
CPU times: user 25.3 s, sys: 2.4 s, total: 27.7 s
Wall time: 6min 50s


HalvingGridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=2786, shuffle=True),
                    estimator=Pipeline(steps=[('counter', CountVectorizer()),
                                              ('tfidf', TfidfTransformer()),
                                              ('clf', LogisticRegression())]),
                    n_jobs=-1,
                    param_grid={'clf__C': [0.001, 0.01, 0.1, 1, 10, 100],
                                'counter__max_df': array([0.5, 0.6, 0.7, 0.8, 0.9]),
                                'counter__min_df': array([0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009]),
                                'counter__ngram_range': ((1, 1), (1, 2)),
                                'tfidf__norm': ('l1', 'l2')},
                    random_state=42, scoring='accuracy', verbose=1)

In [61]:
grid_search_counter_lr.best_params_

{'clf__C': 100,
 'counter__max_df': 0.7,
 'counter__min_df': 0.007,
 'counter__ngram_range': (1, 2),
 'tfidf__norm': 'l2'}

Результаты для LogisticRegression + CountVectorizer:

In [62]:
lr_counter = grid_search_counter_lr.best_estimator_.predict(X_test)
accuracy_lr_counter = accuracy_score(y_test, lr_counter)
recall_lr_counter = recall_score(y_test, lr_counter)

print(classification_report(y_test, lr_counter))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       748
           1       0.99      0.95      0.97       152

    accuracy                           0.99       900
   macro avg       0.99      0.98      0.98       900
weighted avg       0.99      0.99      0.99       900



### DecisionTree + CountVectorizer

In [63]:
pipe_counter_dclf = Pipeline(
    steps=[
        ('counter', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', DecisionTreeClassifier())
    ]
)

In [64]:
parameter_grid = {'counter__max_df': np.arange(0.5, 1, 0.1),
                  'counter__min_df': np.arange(0.001, 0.01, 0.001),
                  "counter__ngram_range": [(1, 2)],
                  "tfidf__norm": ("l1", "l2"),
                  "clf__max_depth": [8, 9, 10, 12, 14, 16],
                  "clf__min_samples_split": [5, 10, 15],
                  "clf__criterion": ['gini', 'entropy'],
                  'clf__max_features': ['sqrt']
}

In [65]:
%%time

grid_search_counter_dclf = HalvingGridSearchCV(
    pipe_counter_dclf,
    param_grid=parameter_grid,
    n_jobs=-1,
    cv=skf,
    scoring='accuracy',
    random_state=42,
)

grid_search_counter_dclf.fit(X_train, y_train);

CPU times: user 3.6 s, sys: 372 ms, total: 3.98 s
Wall time: 1min 4s


HalvingGridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=2786, shuffle=True),
                    estimator=Pipeline(steps=[('counter', CountVectorizer()),
                                              ('tfidf', TfidfTransformer()),
                                              ('clf',
                                               DecisionTreeClassifier())]),
                    n_jobs=-1,
                    param_grid={'clf__criterion': ['gini', 'entropy'],
                                'clf__max_depth': [8, 9, 10, 12, 14, 16],
                                'clf__max_features': ['sqrt'],
                                'clf__min_samples_split': [5, 10, 15],
                                'counter__max_df': [0.9],
                                'counter__min_df': [0.009],
                                'counter__ngram_range': [(1, 2)],
                                'tfidf__norm': ('l1', 'l2')},
                    random_state=42, scoring='accuracy')

In [66]:
grid_search_counter_dclf.best_params_

{'clf__criterion': 'entropy',
 'clf__max_depth': 14,
 'clf__max_features': 'sqrt',
 'clf__min_samples_split': 15,
 'counter__max_df': 0.9,
 'counter__min_df': 0.009,
 'counter__ngram_range': (1, 2),
 'tfidf__norm': 'l1'}

Результаты для DecisionTreeClassifier + CountVectorizer:

In [67]:
dclf_counter = grid_search_counter_dclf.best_estimator_.predict(X_test)
accuracy_dclf_counter = accuracy_score(y_test, dclf_counter)
recall_dclf_counter = recall_score(y_test, dclf_counter)

print(classification_report(y_test, dclf_counter))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       748
           1       0.84      0.74      0.79       152

    accuracy                           0.93       900
   macro avg       0.89      0.85      0.87       900
weighted avg       0.93      0.93      0.93       900



### Naive Bayes + CountVectorizer

In [68]:
pipe_counter_nb = Pipeline(
    steps=[
        ('counter', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', BernoulliNB())
    ]
)

In [69]:
parameter_grid = {'counter__max_df': np.arange(0.5, 1, 0.1),
                  'counter__min_df': np.arange(0.005, 0.01, 0.001),
                  "counter__ngram_range": ((1, 1), (1, 2)),
                  "tfidf__norm": ("l1", "l2"),
                  'clf__alpha': [0.1, 0.5, 1.0, 2.0, 5.0],
                  'clf__fit_prior': [True, False]
}

In [70]:
%%time

grid_search_counter_nb = HalvingGridSearchCV(
    pipe_counter_nb,
    param_grid=parameter_grid,
    n_jobs=-1,
    cv=skf,
    scoring='accuracy',
    random_state=42,
)

grid_search_counter_nb.fit(X_train, y_train);

CPU times: user 18.1 s, sys: 1.63 s, total: 19.7 s
Wall time: 3min 43s


HalvingGridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=2786, shuffle=True),
                    estimator=Pipeline(steps=[('counter', CountVectorizer()),
                                              ('tfidf', TfidfTransformer()),
                                              ('clf', BernoulliNB())]),
                    n_jobs=-1,
                    param_grid={'clf__alpha': [0.1, 0.5, 1.0, 2.0, 5.0],
                                'clf__fit_prior': [True, False],
                                'counter__max_df': array([0.5, 0.6, 0.7, 0.8, 0.9]),
                                'counter__min_df': array([0.005, 0.006, 0.007, 0.008, 0.009]),
                                'counter__ngram_range': ((1, 1), (1, 2)),
                                'tfidf__norm': ('l1', 'l2')},
                    random_state=42, scoring='accuracy')

In [71]:
grid_search_counter_nb.best_params_

{'clf__alpha': 0.1,
 'clf__fit_prior': False,
 'counter__max_df': 0.5,
 'counter__min_df': 0.007,
 'counter__ngram_range': (1, 1),
 'tfidf__norm': 'l1'}

Результаты для Naive Bayes + CountVectorizer:

In [72]:
nb_counter = grid_search_counter_nb.best_estimator_.predict(X_test)
accuracy_nb_counter = accuracy_score(y_test, nb_counter)
recall_nb_counter = recall_score(y_test, nb_counter)

print(classification_report(y_test, nb_counter))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       748
           1       0.89      0.82      0.85       152

    accuracy                           0.95       900
   macro avg       0.93      0.90      0.91       900
weighted avg       0.95      0.95      0.95       900



Подбор гипер-параметров через пайплайн для векторизации при помощи TfidfVectorizer и моделей:

### LR + TfidfVectorizer


In [73]:
pipe_tf_lr = Pipeline(
    steps=[
        ('tfidf', TfidfVectorizer()),
        ('clf', LogisticRegression())
    ]
)

In [74]:
parameter_grid = {'tfidf__max_df': np.arange(0.5, 1, 0.1),
                  'tfidf__min_df': np.arange(0.005, 0.01, 0.001),
                  "tfidf__ngram_range": ((1, 1), (1, 2)),
                  "tfidf__norm": ("l1", "l2"),
                  "clf__C": [0.001, 0.01, 0.1, 1, 10, 100]
}

In [75]:
%%time

grid_search_tf_lr = HalvingGridSearchCV(
    pipe_tf_lr,
    param_grid=parameter_grid,
    n_jobs=-1,
    cv=skf,
    scoring='accuracy',
    random_state=42,
)

grid_search_tf_lr.fit(X_train, y_train);

CPU times: user 13.4 s, sys: 1.2 s, total: 14.6 s
Wall time: 3min 28s


HalvingGridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=2786, shuffle=True),
                    estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf', LogisticRegression())]),
                    n_jobs=-1,
                    param_grid={'clf__C': [0.001, 0.01, 0.1, 1, 10, 100],
                                'tfidf__max_df': array([0.5, 0.6, 0.7, 0.8, 0.9]),
                                'tfidf__min_df': array([0.005, 0.006, 0.007, 0.008, 0.009]),
                                'tfidf__ngram_range': ((1, 1), (1, 2)),
                                'tfidf__norm': ('l1', 'l2')},
                    random_state=42, scoring='accuracy')

In [76]:
grid_search_tf_lr.best_params_

{'clf__C': 100,
 'tfidf__max_df': 0.7,
 'tfidf__min_df': 0.007,
 'tfidf__ngram_range': (1, 2),
 'tfidf__norm': 'l2'}

Результаты для LogisticRegression + TfidfVectorizer:

In [77]:
lr_tf = grid_search_tf_lr.best_estimator_.predict(X_test)
accuracy_lr_tf = accuracy_score(y_test, lr_tf)
recall_lr_tf = recall_score(y_test, lr_tf)

print(classification_report(y_test, lr_tf))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       748
           1       0.99      0.95      0.97       152

    accuracy                           0.99       900
   macro avg       0.99      0.98      0.98       900
weighted avg       0.99      0.99      0.99       900



### DecisionTree + TfidfVectorizer



In [91]:
pipe_tf_dclf = Pipeline(
    steps=[
        ('tfidf', TfidfVectorizer()),
        ('clf', DecisionTreeClassifier())
    ]
)

In [92]:
parameter_grid = {'tfidf__max_df': np.arange(0.5, 1, 0.1),
                  'tfidf__min_df': np.arange(0.005, 0.01, 0.001),
                  "tfidf__ngram_range": ((1, 1), (1, 2)),
                  "tfidf__norm": ("l1", "l2"),
                  "clf__max_depth": [8, 9, 10, 12, 14, 16],
                  "clf__min_samples_split": [5, 10, 15],
                  "clf__criterion": ['gini'],
                  'clf__max_features': ['sqrt', 'log2']
}

⬇ Не запускать, работает 21 минуту 27 секунд



In [94]:
%%time

grid_search_tf_dclf = HalvingGridSearchCV(
    pipe_tf_dclf,
    param_grid=parameter_grid,
    n_jobs=-1,
    cv=skf,
    scoring='accuracy',
    random_state=42,
)

grid_search_tf_dclf.fit(X_train, y_train);

CPU times: user 54.7 s, sys: 4.63 s, total: 59.4 s
Wall time: 14min 23s


HalvingGridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=2786, shuffle=True),
                    estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf',
                                               DecisionTreeClassifier())]),
                    n_jobs=-1,
                    param_grid={'clf__criterion': ['gini'],
                                'clf__max_depth': [8, 9, 10, 12, 14, 16],
                                'clf__max_features': ['sqrt', 'log2'],
                                'clf__min_samples_split': [5, 10, 15],
                                'tfidf__max_df': array([0.5, 0.6, 0.7, 0.8, 0.9]),
                                'tfidf__min_df': array([0.005, 0.006, 0.007, 0.008, 0.009]),
                                'tfidf__ngram_range': ((1, 1), (1, 2)),
                                'tfidf__norm': ('l1', 'l2')},
                    random_state=42, scoring='recall')

In [95]:
grid_search_tf_dclf.best_params_

{'clf__criterion': 'gini',
 'clf__max_depth': 16,
 'clf__max_features': 'sqrt',
 'clf__min_samples_split': 5,
 'tfidf__max_df': 0.5,
 'tfidf__min_df': 0.006,
 'tfidf__ngram_range': (1, 1),
 'tfidf__norm': 'l2'}

Результаты для DecisionTree + TfidfVectorizer:

In [96]:
dclf_tf = grid_search_tf_dclf.best_estimator_.predict(X_test)
accuracy_dclf_tf = accuracy_score(y_test, dclf_tf)
recall_dclf_tf = recall_score(y_test, dclf_tf)

print(classification_report(y_test, dclf_tf))

              precision    recall  f1-score   support

           0       0.92      0.97      0.95       748
           1       0.83      0.61      0.70       152

    accuracy                           0.91       900
   macro avg       0.88      0.79      0.82       900
weighted avg       0.91      0.91      0.91       900



### Naive Bayes + TfidfVectorizer



In [83]:
pipe_tf_nb = Pipeline(
    steps=[
        ('tfidf', TfidfVectorizer()),
        ('clf', BernoulliNB())
    ]
)

In [84]:
parameter_grid = {'tfidf__max_df': np.arange(0.5, 1, 0.1),
                  'tfidf__min_df': np.arange(0.005, 0.01, 0.001),
                  "tfidf__ngram_range": ((1, 1), (1, 2)),
                  "tfidf__norm": ("l1", "l2"),
                  'clf__alpha': [0.1, 0.5, 1.0, 2.0, 5.0],
                  'clf__fit_prior': [True, False]
}

In [85]:
%%time

grid_search_tf_nb = HalvingGridSearchCV(
    pipe_tf_nb,
    param_grid=parameter_grid,
    n_jobs=-1,
    cv=skf,
    scoring='accuracy',
    random_state=42,
)

grid_search_tf_nb.fit(X_train, y_train);

CPU times: user 17.3 s, sys: 1.57 s, total: 18.8 s
Wall time: 3min 39s


HalvingGridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=2786, shuffle=True),
                    estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf', BernoulliNB())]),
                    n_jobs=-1,
                    param_grid={'clf__alpha': [0.1, 0.5, 1.0, 2.0, 5.0],
                                'clf__fit_prior': [True, False],
                                'tfidf__max_df': array([0.5, 0.6, 0.7, 0.8, 0.9]),
                                'tfidf__min_df': array([0.005, 0.006, 0.007, 0.008, 0.009]),
                                'tfidf__ngram_range': ((1, 1), (1, 2)),
                                'tfidf__norm': ('l1', 'l2')},
                    random_state=42, scoring='accuracy')

In [86]:
grid_search_tf_nb.best_params_

{'clf__alpha': 0.1,
 'clf__fit_prior': False,
 'tfidf__max_df': 0.5,
 'tfidf__min_df': 0.007,
 'tfidf__ngram_range': (1, 1),
 'tfidf__norm': 'l1'}

Результаты для Naive Bayes + TfidfVectorizer:

In [87]:
nb_tf = grid_search_tf_nb.best_estimator_.predict(X_test)
accuracy_nb_tf = accuracy_score(y_test, nb_tf)
recall_nb_tf = recall_score(y_test, nb_tf)

print(classification_report(y_test, nb_tf))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       748
           1       0.89      0.82      0.85       152

    accuracy                           0.95       900
   macro avg       0.93      0.90      0.91       900
weighted avg       0.95      0.95      0.95       900



## Сравнение моделей

Сравним все полученные модели более наглядно:

In [97]:
print("CountVectorizer")
print("")
print("Accuracy")
print(f'LogisticRegression: {round(accuracy_lr_counter, 4)}, DecisionTreeClasifier: {round(accuracy_dclf_counter, 4)}, Naive Bayes: {round(accuracy_nb_counter, 4)}')
print("")
print("Recall")
print(f'LogisticRegression: {round(recall_lr_counter, 4)}, DecisionTreeClasifier: {round(recall_dclf_counter, 4)}, Naive Bayes: {round(recall_nb_counter, 4)}')

CountVectorizer

Accuracy
LogisticRegression: 0.99, DecisionTreeClasifier: 0.9322, Naive Bayes: 0.9522

Recall
LogisticRegression: 0.9539, DecisionTreeClasifier: 0.7368, Naive Bayes: 0.8158


In [89]:
print("TfidfVectorizer")
print("")
print("Accuracy")
print(f'LogisticRegression: {round(accuracy_lr_tf, 4)}, DecisionTreeClasifier: {round(accuracy_dclf_tf, 4)}, Naive Bayes: {round(accuracy_nb_tf, 4)}')
print("")
print("Recall")
print(f'LogisticRegression: {round(recall_lr_tf, 4)}, DecisionTreeClasifier: {round(recall_dclf_tf, 4)}, Naive Bayes: {round(recall_nb_tf, 4)}')

TfidfVectorizer

Accuracy
LogisticRegression: 0.99, DecisionTreeClasifier: 0.9189, Naive Bayes: 0.9522

Recall
LogisticRegression: 0.9539, DecisionTreeClasifier: 0.6118, Naive Bayes: 0.8158


In [6]:
end_time = time.time()

execution_time = end_time - start_time
print(f"Время выполнения ноутбука: {execution_time // 60:.0f} минут(ы) {(execution_time - (execution_time // 60) * 60):.0f} секунд(ы)")

Время выполнения ноутбука: 1.00 минут 55.884095668792725
